In [ ]:
import os
import yaml

# Pobranie katalogu bazowego (wyjście poziom wyżej z `notebooks/`)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Tworzenie poprawnej ścieżki do pliku `config.yaml`
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")

# Wczytanie pliku YAML
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

bot_token = config["telegram"]["bot_token"]
chat_id = config["telegram"]["chat_id"]

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import asyncio
import nest_asyncio

from bot.data.stream.multiple_tickers_data_source import MultipleTickersDataSource
from bot.data.stream.binance_stream_manager import BinanceStreamManager
# from data.binance_data_fetcher import BinanceDataFetcher
# from core.candle_store import CandleStore
from bot.core.strategy03 import Strategy03
from bot.core.strategy04 import Strategy04
from bot.core.strategy05 import Strategy05
from bot.core.executor import Executor
from bot.core.engine import Engine

# Konfiguracja parametrów
tickers = ["BTCUSDT", "ETHUSDT", "DOTUSDT", "AVAXUSDT", "SOLUSDT", "XRPUSDT", "ADAUSDT", "PEPEUSDT", "LINKUSDT"]
source_interval = "1m"
intervals = ['1m', '5m', '15m', '1h', '4h', '1d'] 
number_candles = 50

# Tworzenie komponentów
# binance_data_fetcher = BinanceDataFetcher()
data_source = MultipleTickersDataSource(tickers=tickers, interval=source_interval, only_closed=False)

# Strategia z domyślnymi kolumnami: 'time_utc', 'open', 'close', 'is_closed'
# strategy03 = Strategy03(tickers, intervals, number_candles, display_columns=['time_utc', 'open', 'close', 'volume', 'is_closed'])
# strategy04 = Strategy04(tickers, intervals, number_candles, display_columns=['time_utc', 'open', 'close', 'volume', 'is_closed'])
strategy05 = Strategy05(tickers, intervals, number_candles, display_columns=['time_utc', 'open', 'close', 'volume', 'is_closed'])
executor = Executor(bot_token=bot_token, chat_id=chat_id)
engine = Engine(data_source, executor)
# engine.add_strategy(strategy03)
# engine.add_strategy(strategy04)
engine.add_strategy(strategy05)

# Zastosowanie nest_asyncio dla Jupyter Notebook lub środowisk interaktywnych
nest_asyncio.apply()

# Główna funkcja asynchroniczna
async def main():
    engine.setup()
    await engine.start()

# Uruchomienie
await main()